In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
import os
import time
import pygame


# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Define get_facial_metrics function
def get_facial_metrics(landmarks, frame_shape):
    metrics = {}
    ih, iw = frame_shape[:2]

    # Convert landmarks to pixel coordinates
    landmark_points = []
    for landmark in landmarks.landmark:
        x, y = int(landmark.x * iw), int(landmark.y * ih)
        landmark_points.append((x, y))
    
    landmark_points = np.array(landmark_points)

    # Jawline measurements (approximate jaw points)
    jaw_indices = [152, 176, 148, 149, 150, 136, 172, 58, 132, 93, 234]
    jaw_points = landmark_points[jaw_indices]
    metrics['jaw_length'] = np.sum(np.linalg.norm(jaw_points[1:] - jaw_points[:-1], axis=1))

    # Eyebrow distances
    left_eyebrow = landmark_points[[55, 65, 52]]
    right_eyebrow = landmark_points[[285, 295, 282]]
    metrics['eyebrow_asymmetry'] = np.mean(np.abs(left_eyebrow[:,1] - right_eyebrow[:,1]))

    # Mouth measurements
    mouth_outer = landmark_points[[61, 291, 0, 17]]  # Left, Right, Top, Bottom
    metrics['mouth_width'] = np.linalg.norm(mouth_outer[1] - mouth_outer[0])
    metrics['mouth_height'] = np.linalg.norm(mouth_outer[3] - mouth_outer[2])
    metrics['mouth_aspect_ratio'] = metrics['mouth_width'] / (metrics['mouth_height'] + 1e-6)

    # Eye aspect ratio
    def eye_aspect_ratio(eye_points):
        A = np.linalg.norm(eye_points[1] - eye_points[5])
        B = np.linalg.norm(eye_points[2] - eye_points[4])
        C = np.linalg.norm(eye_points[0] - eye_points[3])
        return (A + B) / (2.0 * C + 1e-6)
    
    left_eye = landmark_points[[33, 160, 158, 133, 153, 144]]
    right_eye = landmark_points[[362, 385, 387, 263, 373, 380]]
    metrics['left_eye_ratio'] = eye_aspect_ratio(left_eye)
    metrics['right_eye_ratio'] = eye_aspect_ratio(right_eye)

    # Symmetry error
    left_face = landmark_points[[33, 133, 144, 55, 65]]
    right_face = landmark_points[[362, 263, 380, 285, 295]]
    symmetry_error = np.mean(np.abs(left_face[:,0] - (iw - right_face[:,0])))
    metrics['symmetry_error'] = symmetry_error

    return metrics

# Function to process dataset
def process_dataset(dataset_path, output_csv="facial_metrics.csv"):
    data = []
    allowed_emotions = {"angry", "happy", "neutral", "sad", "surprise"}
    emotions = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d)) and d.lower() in allowed_emotions]
    
    for emotion in emotions:
        emotion_dir = os.path.join(dataset_path, emotion)
        print(f"Processing {emotion}...")
        
        for img_name in tqdm(os.listdir(emotion_dir)):
            img_path = os.path.join(emotion_dir, img_name)
            image = cv2.imread(img_path)
            
            if image is None:
                continue
                
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb_image)
            
            if results.multi_face_landmarks:
                metrics = get_facial_metrics(results.multi_face_landmarks[0], image.shape)
                metrics["label"] = emotion
                data.append(metrics)
    
    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    return df


# Function to train model
def train_emotion_model(csv_path, model_save_path="emotion_modelandfacial.pkl"):
    df = pd.read_csv(csv_path)
    
    le = LabelEncoder()
    y = le.fit_transform(df["label"])
    X = df.drop("label", axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    model = SVC(kernel='rbf', C=10, gamma=0.01, probability=True)
    model.fit(X_train, y_train)
    
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    print(f"Training Accuracy: {train_acc:.2f}")
    print(f"Test Accuracy: {test_acc:.2f}")
    
    with open(model_save_path, 'wb') as f:
        pickle.dump((model, scaler, le), f)
    
    return model

# Function for real-time detection
import pygame
import time
import threading

def play_music(music_path):
    try:
        pygame.mixer.init()
        pygame.mixer.music.load(music_path)
        pygame.mixer.music.play()
        print("Music started...")

        while pygame.mixer.music.get_busy():
            time.sleep(1)
    except Exception as e:
        print(f"Music playback failed: {e}")


def real_time_emotion_recognition(model_path="emotion_modelandfacial.pkl", music_path=r"E:\asd project 2\funny-kids-cartoon-background-music-333104.mp3"):
    with open(model_path, 'rb') as f:
        model, scaler, le = pickle.load(f)
    
    cap = cv2.VideoCapture(0)
    face_mesh_real_time = mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    # Initialize pygame mixer
    pygame.mixer.init()
    sad_sound = pygame.mixer.Sound(music_path)
    sad_sound.set_volume(0.5)

    last_music_play_time = 0
    cooldown = 10  # seconds
    music_playing = False

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh_real_time.process(rgb_frame)
        
        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0]
            metrics = get_facial_metrics(landmarks, frame.shape)
            
            features = np.array(list(metrics.values())).reshape(1, -1)
            features = scaler.transform(features)
            
            emotion_idx = model.predict(features)[0]
            emotion = le.inverse_transform([emotion_idx])[0]

            cv2.putText(frame, f"Emotion: {emotion}", (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            current_time = time.time()

            if emotion == "sad":
                if not music_playing and (current_time - last_music_play_time > cooldown):
                    sad_sound.play()
                    last_music_play_time = current_time
                    music_playing = True
            else:
                if music_playing:
                    sad_sound.stop()
                    music_playing = False
        
        cv2.imshow("Emotion Recognition", frame)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    pygame.mixer.quit()

         

# Example workflow:
if __name__ == "__main__":
    #Step 1: Process your dataset
    process_dataset(r"E:\asd project 2\dataset3")
    
    # Step 2: Train the model
    train_emotion_model("facial_metrics.csv")
    
    # Step 3: Run real-time detection
    real_time_emotion_recognition()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Processing angry...


100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:01<00:00, 21.28it/s]


Processing happy...


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [00:02<00:00, 23.78it/s]


Processing neutral...


...

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:03<00:00, 23.47it/s]


Processing surprise...


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:01<00:00, 22.55it/s]


Training Accuracy: 0.72
Test Accuracy: 0.76


E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but 

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
import os
import time
import pygame
import webbrowser

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Define get_facial_metrics
def get_facial_metrics(landmarks, frame_shape):
    metrics = {}
    ih, iw = frame_shape[:2]
    landmark_points = [(int(lm.x * iw), int(lm.y * ih)) for lm in landmarks.landmark]
    landmark_points = np.array(landmark_points)

    jaw_indices = [152, 176, 148, 149, 150, 136, 172, 58, 132, 93, 234]
    jaw_points = landmark_points[jaw_indices]
    metrics['jaw_length'] = np.sum(np.linalg.norm(jaw_points[1:] - jaw_points[:-1], axis=1))

    left_eyebrow = landmark_points[[55, 65, 52]]
    right_eyebrow = landmark_points[[285, 295, 282]]
    metrics['eyebrow_asymmetry'] = np.mean(np.abs(left_eyebrow[:,1] - right_eyebrow[:,1]))

    mouth_outer = landmark_points[[61, 291, 0, 17]]
    metrics['mouth_width'] = np.linalg.norm(mouth_outer[1] - mouth_outer[0])
    metrics['mouth_height'] = np.linalg.norm(mouth_outer[3] - mouth_outer[2])
    metrics['mouth_aspect_ratio'] = metrics['mouth_width'] / (metrics['mouth_height'] + 1e-6)

    def eye_aspect_ratio(eye_points):
        A = np.linalg.norm(eye_points[1] - eye_points[5])
        B = np.linalg.norm(eye_points[2] - eye_points[4])
        C = np.linalg.norm(eye_points[0] - eye_points[3])
        return (A + B) / (2.0 * C + 1e-6)

    left_eye = landmark_points[[33, 160, 158, 133, 153, 144]]
    right_eye = landmark_points[[362, 385, 387, 263, 373, 380]]
    metrics['left_eye_ratio'] = eye_aspect_ratio(left_eye)
    metrics['right_eye_ratio'] = eye_aspect_ratio(right_eye)

    left_face = landmark_points[[33, 133, 144, 55, 65]]
    right_face = landmark_points[[362, 263, 380, 285, 295]]
    symmetry_error = np.mean(np.abs(left_face[:,0] - (iw - right_face[:,0])))
    metrics['symmetry_error'] = symmetry_error

    return metrics

# Dataset processor
def process_dataset(dataset_path, output_csv="facial_metrics.csv"):
    data = []
    allowed_emotions = {"angry", "happy", "neutral", "sad", "surprise"}
    emotions = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d)) and d.lower() in allowed_emotions]
    
    for emotion in emotions:
        emotion_dir = os.path.join(dataset_path, emotion)
        print(f"Processing {emotion}...")
        
        for img_name in tqdm(os.listdir(emotion_dir)):
            img_path = os.path.join(emotion_dir, img_name)
            image = cv2.imread(img_path)
            if image is None:
                continue
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb_image)
            if results.multi_face_landmarks:
                metrics = get_facial_metrics(results.multi_face_landmarks[0], image.shape)
                metrics["label"] = emotion
                data.append(metrics)
    
    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    return df

# Model trainer
def train_emotion_model(csv_path, model_save_path="emotion_modelandfacial.pkl"):
    df = pd.read_csv(csv_path)
    le = LabelEncoder()
    y = le.fit_transform(df["label"])
    X = df.drop("label", axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = SVC(kernel='rbf', C=10, gamma=0.01, probability=True)
    model.fit(X_train, y_train)

    print(f"Training Accuracy: {model.score(X_train, y_train):.2f}")
    print(f"Test Accuracy: {model.score(X_test, y_test):.2f}")

    with open(model_save_path, 'wb') as f:
        pickle.dump((model, scaler, le), f)
    
    return model

# Real-time detector with SAD and ANGRY logic
def real_time_emotion_recognition(model_path="emotion_modelandfacial.pkl", music_path=r"E:\asd project 2\funny-kids-cartoon-background-music-333104.mp3", youtube_link="https://www.youtube.com/watch?v=dQw4w9WgXcQ"):
    with open(model_path, 'rb') as f:
        model, scaler, le = pickle.load(f)
    
    cap = cv2.VideoCapture(0)
    face_mesh_rt = mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    pygame.mixer.init()
    sad_sound = pygame.mixer.Sound(music_path)
    sad_sound.set_volume(0.5)

    last_music_play_time = 0
    cooldown = 10
    music_playing = False
    angry_start_time = None
    youtube_opened = False

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh_rt.process(rgb_frame)

        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0]
            metrics = get_facial_metrics(landmarks, frame.shape)
            features = np.array(list(metrics.values())).reshape(1, -1)
            features = scaler.transform(features)

            emotion_idx = model.predict(features)[0]
            emotion = le.inverse_transform([emotion_idx])[0].lower()

            cv2.putText(frame, f"Emotion: {emotion}", (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            current_time = time.time()

            # Sad logic
            if emotion == "sad":
                if not music_playing and (current_time - last_music_play_time > cooldown):
                    sad_sound.play()
                    last_music_play_time = current_time
                    music_playing = True
            else:
                if music_playing:
                    sad_sound.stop()
                    music_playing = False

            # Angry logic
            if emotion == "angry":
                print("Angry detected.")
                if angry_start_time is None:
                    angry_start_time = current_time
                elif not youtube_opened and (current_time - angry_start_time > 10):
                    print("Opening YouTube due to prolonged anger.")
                    webbrowser.open(youtube_link)
                    youtube_opened = True
            else:
                angry_start_time = None
                youtube_opened = False

        cv2.imshow("Emotion Recognition", frame)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    pygame.mixer.quit()

# Run full workflow
if __name__ == "__main__":
    dataset_path = r"E:\asd project 2\dataset3"
    music_file = r"E:\asd project 2\funny-kids-cartoon-background-music-333104.mp3"
    youtube_link = "https://youtu.be/5oH9Nr3bKfw?feature=shared" # Replace with your actual video

    process_dataset(dataset_path)
    train_emotion_model("facial_metrics.csv")
    real_time_emotion_recognition("emotion_modelandfacial.pkl", music_file, youtube_link)


Processing angry...


100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:01<00:00, 34.56it/s]


Processing happy...


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [00:01<00:00, 36.64it/s]


Processing neutral...


...

E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but 

Angry detected.
Angry detected.
Angry detected.
Angry detected.
Angry detected.


E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but 

Angry detected.
Angry detected.


E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but 

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
import os
import time
import pygame
import webbrowser

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Define get_facial_metrics
def get_facial_metrics(landmarks, frame_shape):
    metrics = {}
    ih, iw = frame_shape[:2]
    landmark_points = [(int(lm.x * iw), int(lm.y * ih)) for lm in landmarks.landmark]
    landmark_points = np.array(landmark_points)

    jaw_indices = [152, 176, 148, 149, 150, 136, 172, 58, 132, 93, 234]
    jaw_points = landmark_points[jaw_indices]
    metrics['jaw_length'] = np.sum(np.linalg.norm(jaw_points[1:] - jaw_points[:-1], axis=1))

    left_eyebrow = landmark_points[[55, 65, 52]]
    right_eyebrow = landmark_points[[285, 295, 282]]
    metrics['eyebrow_asymmetry'] = np.mean(np.abs(left_eyebrow[:,1] - right_eyebrow[:,1]))

    mouth_outer = landmark_points[[61, 291, 0, 17]]
    metrics['mouth_width'] = np.linalg.norm(mouth_outer[1] - mouth_outer[0])
    metrics['mouth_height'] = np.linalg.norm(mouth_outer[3] - mouth_outer[2])
    metrics['mouth_aspect_ratio'] = metrics['mouth_width'] / (metrics['mouth_height'] + 1e-6)

    def eye_aspect_ratio(eye_points):
        A = np.linalg.norm(eye_points[1] - eye_points[5])
        B = np.linalg.norm(eye_points[2] - eye_points[4])
        C = np.linalg.norm(eye_points[0] - eye_points[3])
        return (A + B) / (2.0 * C + 1e-6)

    left_eye = landmark_points[[33, 160, 158, 133, 153, 144]]
    right_eye = landmark_points[[362, 385, 387, 263, 373, 380]]
    metrics['left_eye_ratio'] = eye_aspect_ratio(left_eye)
    metrics['right_eye_ratio'] = eye_aspect_ratio(right_eye)

    left_face = landmark_points[[33, 133, 144, 55, 65]]
    right_face = landmark_points[[362, 263, 380, 285, 295]]
    symmetry_error = np.mean(np.abs(left_face[:,0] - (iw - right_face[:,0])))
    metrics['symmetry_error'] = symmetry_error

    return metrics

# Dataset processor
def process_dataset(dataset_path, output_csv="facial_metrics.csv"):
    data = []
    allowed_emotions = {"angry", "happy", "neutral", "sad", "surprise"}
    emotions = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d)) and d.lower() in allowed_emotions]
    
    for emotion in emotions:
        emotion_dir = os.path.join(dataset_path, emotion)
        print(f"Processing {emotion}...")
        
        for img_name in tqdm(os.listdir(emotion_dir)):
            img_path = os.path.join(emotion_dir, img_name)
            image = cv2.imread(img_path)
            if image is None:
                continue
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb_image)
            if results.multi_face_landmarks:
                metrics = get_facial_metrics(results.multi_face_landmarks[0], image.shape)
                metrics["label"] = emotion
                data.append(metrics)
    
    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    return df

# Model trainer
def train_emotion_model(csv_path, model_save_path="emotion_modelandfacial.pkl"):
    df = pd.read_csv(csv_path)
    le = LabelEncoder()
    y = le.fit_transform(df["label"])
    X = df.drop("label", axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = SVC(kernel='rbf', C=10, gamma=0.01, probability=True)
    model.fit(X_train, y_train)

    print(f"Training Accuracy: {model.score(X_train, y_train):.2f}")
    print(f"Test Accuracy: {model.score(X_test, y_test):.2f}")

    with open(model_save_path, 'wb') as f:
        pickle.dump((model, scaler, le), f)
    
    return model

# Real-time detector with SAD and ANGRY logic
def real_time_emotion_recognition(model_path="emotion_modelandfacial.pkl", music_path=r"E:\asd project 2\funny-kids-cartoon-background-music-333104.mp3", youtube_link="https://www.youtube.com/watch?v=dQw4w9WgXcQ"):
    with open(model_path, 'rb') as f:
        model, scaler, le = pickle.load(f)
    
    cap = cv2.VideoCapture(0)
    face_mesh_rt = mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    pygame.mixer.init()
    sad_sound = pygame.mixer.Sound(music_path)
    sad_sound.set_volume(0.5)

    last_music_play_time = 0
    cooldown = 10
    music_playing = False
    angry_start_time = None
    youtube_opened = False

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh_rt.process(rgb_frame)

        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0]
            ih, iw = frame.shape[:2]
            landmark_points = [(int(lm.x * iw), int(lm.y * ih)) for lm in landmarks.landmark]
            xs = [pt[0] for pt in landmark_points]
            ys = [pt[1] for pt in landmark_points]
    
    # Draw bounding box around face
            x_min, x_max = min(xs), max(xs)
            y_min, y_max = min(ys), max(ys)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

    # Optional: draw label background
            label = "Detecting..."
            metrics = get_facial_metrics(landmarks, frame.shape)
            features = np.array(list(metrics.values())).reshape(1, -1)
            features = scaler.transform(features)

            emotion_idx = model.predict(features)[0]
            emotion = le.inverse_transform([emotion_idx])[0].lower()
            label = f"Emotion: {emotion}"
            
    # Label text box
            cv2.rectangle(frame, (x_min, y_min - 30), (x_min + 250, y_min), (255, 0, 0), -1)
            cv2.putText(frame, label, (x_min + 5, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        
        
            features = np.array(list(metrics.values())).reshape(1, -1)
            features = scaler.transform(features)

            
            emotion_idx = model.predict(features)[0]
            emotion = le.inverse_transform([emotion_idx])[0].lower()

          

            current_time = time.time()

            # Sad logic
            if emotion == "sad":
                if not music_playing and (current_time - last_music_play_time > cooldown):
                    sad_sound.play()
                    last_music_play_time = current_time
                    music_playing = True
            else:
                if music_playing:
                    sad_sound.stop()
                    music_playing = False

            # Angry logic
            if emotion == "angry":
                print("Angry detected.")
                if angry_start_time is None:
                    angry_start_time = current_time
                elif not youtube_opened and (current_time - angry_start_time > 10):
                    print("Opening YouTube due to prolonged anger.")
                    webbrowser.open(youtube_link)
                    youtube_opened = True
            else:
                angry_start_time = None
                youtube_opened = False
            h, w, _ = frame.shape
            display_lines = [
                f"Jawline: {metrics['jaw_length']:.2f}",
                f"Mouth Open: {metrics['mouth_height']:.2f}",
                f"Mouth Width: {metrics['mouth_width']:.2f}",
                f"Mouth Ratio: {metrics['mouth_aspect_ratio']:.2f}",
                f"Left Eye Ratio: {metrics['left_eye_ratio']:.2f}",
                f"Right Eye Ratio: {metrics['right_eye_ratio']:.2f}",
                f"Eyebrow Asymmetry: {metrics['eyebrow_asymmetry']:.2f}",
                f"Symmetry Error: {metrics['symmetry_error']:.2f}"
            ]

            # Display each line at the bottom-right
            y_start = h - 180  # Adjust starting height
            for i, line in enumerate(display_lines):
                y = y_start + i * 25
                text_size, _ = cv2.getTextSize(line, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 3)
                x = w - text_size[0] - 10
                cv2.putText(frame, line, (x, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1)
                
        cv2.imshow("Emotion Recognition", frame)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    pygame.mixer.quit()


if __name__ == "__main__":
    dataset_path = r"E:\asd project 2\dataset3"
    music_file = r"E:\asd project 2\funny-kids-cartoon-background-music-333104.mp3"
    youtube_link = "https://youtu.be/5oH9Nr3bKfw?feature=shared" # Replace with your actual video

    process_dataset(dataset_path)
    train_emotion_model("facial_metrics.csv")
    real_time_emotion_recognition("emotion_modelandfacial.pkl", music_file, youtube_link)


Processing angry...


100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 85.67it/s]


Processing happy...


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [00:00<00:00, 97.46it/s]


Processing neutral...


...

E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but 

Angry detected.
Angry detected.
Angry detected.
Angry detected.
Angry detected.


E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but 

Angry detected.


E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
E:\emotionrecognition\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but 